# Simple Dash Plotly Dashboard

In [1]:
import pandas as pd
import dash
from dash import dcc
import plotly.graph_objects as go
from dash import html

In [ ]:
###

In [97]:

#### Organise labels for drop down

def get_station(filename):
    df = pd.read_csv(filename)
    # list_station = 

    dict_list = []
    for i in zip(df.station,df.alias):
        dict_list.append({'label': i[1], 'value': i[0]})

    return dict_list


stations = get_station('Dauerzaehlstellen.csv')


In [98]:
stations

[{'label': 'Airport N 1', 'value': 1013},
 {'label': 'Airport N 2', 'value': 1017},
 {'label': 'Airport N 3', 'value': 1018},
 {'label': 'Airport E', 'value': 1022},
 {'label': 'Airport S', 'value': 1035},
 {'label': 'HH West', 'value': 1054},
 {'label': 'Eimsbüttel', 'value': 1061},
 {'label': 'Eimsbüttel 2', 'value': 1077},
 {'label': 'Barmbek', 'value': 1081},
 {'label': 'HH East', 'value': 1097},
 {'label': 'City Center', 'value': 1114},
 {'label': 'Speicherstadt', 'value': 1123},
 {'label': 'HafenCity', 'value': 1134},
 {'label': 'Elbtunnel feeder', 'value': 1145},
 {'label': 'Harbour', 'value': 1154},
 {'label': 'Motorway feeder', 'value': 1173},
 {'label': 'Interchange HH South ', 'value': 1196},
 {'label': 'Wilhelmsburg', 'value': 1212},
 {'label': 'Harburg', 'value': 1221}]

In [2]:
LEGEND = ["POSITIVE", "NEGATIVE"]
SCORES = [0.5, 0.2]

def get_figure(legend, scores):
    return go.Figure(
        [go.Bar(x=legend, y=scores)],
        layout=go.Layout(template="simple_white"),
    )


fig = get_figure(LEGEND, SCORES)

In [3]:
fig.show()

In [12]:
len(df.columns)

53

In [28]:


df = pd.read_csv('Z1061_forecast.csv',parse_dates=True,index_col=0, usecols=range(1,53))
df = df.reset_index()
df = df[['ds','yhat']]
df['Zählstelle'] = 1061



df2 = pd.read_csv('Z1173_forecast.csv',parse_dates=True,index_col=0, usecols=range(1,53))
df2 = df2.reset_index()
df2 = df2[['ds','yhat']]
df2['Zählstelle'] = 1173


df = pd.concat([df,df2])


In [29]:
df.to_csv('compiled_predictions.csv',index=False)

In [31]:
df = pd.read_csv('Z1061_true.csv',parse_dates=True,index_col=1)
df = df.reset_index()
df = df[['ds','y']]
df['Zählstelle'] = 1061



df2 = pd.read_csv('Z1173_true.csv',parse_dates=True,index_col=1)
df2 = df2.reset_index()
df2 = df2[['ds','y']]
df2['Zählstelle'] = 1173


df = pd.concat([df,df2])


In [32]:
df.to_csv('compiled_trues.csv',index=False)

In [39]:
df = pd.read_csv('compiled_predictions.csv',parse_dates=True)
df2 = pd.read_csv('compiled_trues.csv',parse_dates=True)

In [40]:
df3 = pd.merge(df,df2,how='left',left_on=['Zählstelle','ds'], right_on=['Zählstelle','ds'])

In [41]:
df3.yhat = df3.yhat.round()

In [43]:
df3.to_csv('all_data.csv',index=False)


In [163]:
def shift_time_str(date_str,day_shift):
    return (pd.to_datetime(date_str) + pd.Timedelta(days=day_shift)).strftime('%Y-%m-%d')

In [187]:
plt_data

,ds,yhat,Zählstelle,y
7053,2022-03-02,114929.0,1173,123502.0


In [188]:
mydate = '2022-03-01'
zähl = 1173
date_past = shift_time_str(mydate,-7)
date_future = shift_time_str(mydate,1)

df4 = df3.query(f'(Zählstelle == {zähl}) and (ds >= "{date_past}") and (ds <= "{date_future}")')
traces = []

In [189]:
plt_data = df4[df4.ds <= mydate ]
traces.append(go.Scatter(x=plt_data.ds,
                                 y=plt_data.y,
                                 mode='lines',
                                 opacity=0.7,
                                 name='Past actual',
                                 textposition='bottom center'))

traces.append(go.Scatter(x=plt_data.ds,
                                 y=plt_data.yhat,
                                 mode='lines',
                                 opacity=0.7,
                                 name='Past model fit',
                                 textposition='bottom center'))



In [190]:
plt_data = df4[df4.ds > mydate ]
traces.append(go.Scatter(x=plt_data.ds,
                                 y=plt_data.y,
                                 mode='markers',
                                 opacity=0.7,
                                 name='Tomorrow actual',
                                 textposition='bottom center'))

traces.append(go.Scatter(x=plt_data.ds,
                                 y=plt_data.yhat,
                                 mode='markers',
                                 opacity=0.7,
                                 name='Prediction',
                                 textposition='bottom center'))

In [191]:
def get_figure(traces):
       min_day = min([min(trace.x) for trace in traces])
       max_day = max([max(trace.x) for trace in traces])
       min_day = shift_time_str(min_day,-5)
       max_day = shift_time_str(max_day,5)

       figure = {'data': traces,
              'layout': go.Layout(
                #  colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                #   template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  title={'text': 'Traffic density', 'font': {'color': 'grey'}, 'x': 0.5},
                  xaxis={'range': [min_day,max_day]},
              ),}
       return figure

In [192]:
fig = get_figure(traces)
go.Figure(data=fig)

In [194]:
figure_empty = {
              'layout': go.Layout(
                #  colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                #   template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  title={'text': 'Traffic density', 'font': {'color': 'grey'}, 'x': 0.5},
                  xaxis={'range': ['2022-01-01','2022-12-31']},
              ),}

go.Figure(data=figure_empty)

NameError: name 'min_day' is not defined